# HDF to Geotif

In [9]:
from osgeo import gdal
import os


In [ ]:
def get_hdf_file_list(path):
    """
    Get a list of the hdf files inside the folder
    Parameters:
    - path (str): path of the folder with the resources.

    Returns:
    - File_list (list). list of the resources.
    """
    File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for file in os.listdir(path):
        if file.endswith(".h5"):
            if file not in File_list:
                File_list.append(os.path.join(path,file))
        else:
            pass
    return File_list

def create_folder_if_not_exists(folder_path):
    """
    Create a folder if it doesn't exist.

    Parameters:
    folder_path (str): The path of the folder to be created.
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder created at: {folder_path}")
    else:
        print(f"Folder already exists at: {folder_path}")
        
        
def remove_files_with_format(path, format):
    """
    Create a folder if it doesn't exist.

    Parameters:
    folder_path (str): The path of the folder to be created.
    """
    for file in os.listdir(path):
        if file.endswith(format):
            os.remove(os.path.join(file))

In [ ]:
## List input raster files

input_path = r"C:\Users\admin\Downloads\nightdata"
outputFolder = input_path + "\output_data"

create_folder_if_not_exists(outputFolder)
rasterFiles = get_hdf_file_list(input_path)

In [ ]:
for file in rasterFiles[:]:

    ## Open HDF file
    hdflayer = gdal.Open(file, gdal.GA_ReadOnly)

    #print (hdflayer.GetSubDatasets())

    # Open raster layer
    #hdflayer.GetSubDatasets()[0][0] - for first layer
    #hdflayer.GetSubDatasets()[1][0] - for second layer ...etc
    
    sub_hdf_layer = hdflayer.GetSubDatasets()[0][0]
    rlayer = gdal.Open(sub_hdf_layer, gdal.GA_ReadOnly)
    #outputName = rlayer.GetMetadata_Dict()['long_name']


    """This will be updated depending on the data"""
    #Subset the Long Name
    outputName = sub_hdf_layer[92:]

    outputNameNoSpace = outputName.strip().replace(" ","_").replace("/","_")
    file_name = os.path.basename(file[:-3])
    outputNameFinal = outputNameNoSpace + "_" + file_name + "_BBOX.tif"
    #print(outputNameFinal)

    output_raster = os.path.join(outputFolder, outputNameFinal)

    #collect bounding box coordinates
    HorizontalTileNumber = int(rlayer.GetMetadata_Dict()["HorizontalTileNumber"])
    VerticalTileNumber = int(rlayer.GetMetadata_Dict()["VerticalTileNumber"])
        
    WestBoundCoord = (10*HorizontalTileNumber) - 180
    NorthBoundCoord = 90-(10*VerticalTileNumber)
    EastBoundCoord = WestBoundCoord + 10
    SouthBoundCoord = NorthBoundCoord - 10

    EPSG = "-a_srs EPSG:4326" #WGS84

    translateOptionText = EPSG + " -a_ullr " + str(WestBoundCoord) + " " + str(NorthBoundCoord) + " " + str(EastBoundCoord) + " " + str(SouthBoundCoord)

    translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
    gdal.Translate(output_raster, rlayer, options=translateoptions)

    hdflayer = None
    rlayer = None

    #Display image in QGIS (run it within QGIS python Console) - remove comment to display
    #iface.addRasterLayer(outputRaster, outputNameFinal)



In [ ]:
#delete xml files that are generated after opening the hdf files
remove_files_with_format(input_path, ".xml")
